In [1]:
from glob import glob
import shutil
from PIL import Image
import os
from tqdm import tqdm

In [36]:
file_list=glob('../../result/synth/**/**/*.png')

In [ ]:
for i in tqdm(range(len(file_list))):

    class_name=os.path.dirname(file_list[i]).split('유형')[1]
    carcinoma=os.path.basename(os.path.dirname(os.path.dirname(file_list[i])))
    serial_number=os.path.basename(file_list[i]).split('.png')[0]
    im = Image.open(file_list[i])
    im.save(f"{os.path.dirname(file_list[i])}/NIA_S_{carcinoma}_{class_name}_{serial_number.zfill(6)}.jpeg")
    os.remove(file_list[i])

In [3]:
path_list=glob('../../result/synth/BRID/**/')
for i in range(len(path_list)):
    file_list=glob(path_list[i]+'*.jpeg')
    for j in tqdm(range(len(file_list))):
        shutil.move(file_list[j],file_list[j].replace('_00', '_10'))
        
    

100%|██████████| 916/916 [00:07<00:00, 117.30it/s]
